expecting to import googleNet model

In [ ]:
from torch.optim import SGD
# import model from path
import torchvision.models as models
import torch.nn as nn
model = models.googlenet(pretrained=True)
loss_fn = nn.CrossEntropyLoss()
optimizer=SGD(model.parameters(),lr=0.001)
num_ftrs = model.fc.in_features
model.fc = nn.Sequential(
    nn.Linear(num_ftrs, 4)
)


In [ ]:
from path import train_l , X_train , y_train , y_test ,X_test
import keras
import torch
import time
start = time.time()
EPOCHS =  25
for epoch in range( EPOCHS) :
    # TRAIN  MODEL
    model.train()

    #INITIALIZE THE LOSS BY 0
    loss = 0.00

    for X_l  , y_l in train_l:

        optimizer.zero_grad() # clear the gradients from the previous step
        y_pred = model(X_l) # get the prediction values
        l = loss_fn(y_pred,y_l) # calculate the loss
        l.backward() # execute the backpropagation
        optimizer.step() # adjust the weights (parameters)
        loss += l.item() * len(X_l)
    loss_epoch = loss/len(X_train)

end = time.time()
print("Temps total d'entraînement :", (end - start)/60, "minutes")

In [ ]:
model.eval()
with torch.no_grad() :
    # get the final learned weight
    weigth = model.linear.weigth.item()
    bias = model.linear.bias.item()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns


print("\nGénération des prédictions pour la matrice de confusion...")
y_pred = model.predict(X_test, batch_size=32)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1) if len(y_test.shape) > 1 else y_test

cm = confusion_matrix(y_true_classes, y_pred_classes)

plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar_kws={'label': 'Count'})
plt.title('Matrice de Confusion', fontsize=16, fontweight='bold', pad=20)
plt.ylabel('Classe Réelle', fontsize=12)
plt.xlabel('Classe Prédite', fontsize=12)

In [ ]:
import time
# saving the model
PATH = '../../models/model-'+time.localtime()+'.pth'
# get all the learned parameters
model_weights = model.state_dict()
torch.save(model_weights,PATH)